In [2]:
from utils.processor import BaseSampleProcessor
from utils.chat import Chat

In [3]:
%load_ext autoreload
%autoreload 2

In [26]:
few_shot_messages = []


llm_config={
    "model": "gpt-4o-mini",
    "temperature": 1.2,
    "n":5,
}

prompt = (
    "Provide your best guess for the following multi-hop question. "
    "Provide a step-by-step explanation of your thought process. "
    "Each step should only contain a single new fact and be on a new line. "
    "When enough information is present, give your final answer on a new line with no other words or explanation. "
    "Also provide how confident you are between 0.0 and 1.0 that a given line is correct.\n\n"
    "<format>\n"
    "Step 1: {first reasoning step} Confidence: {confidence for step 1}\n"
    "Step 2: {second reasoning step} Confidence: {confidence for step 2}\n"
    "...\n"
    "Final Answer: {most likely guess, as short as possible; not a complete sentence, just the guess!} "
    "Confidence: {final confidence for your answer}\n"
    "</format>\n"
)

few_shot_messages = [
    (
        "Chairmen of the Bored is an album that references a joke made by a cast member of what series?",
        """Step 1: The phrase "Chairmen of the Bored" is a play on words related to the term "Chairman of the Board." Confidence: 0.36
Step 2: Chairmen of the Bored" is an album title associated with a musical group, most likely a parody group. Confidence: 0.32
Step 3: The musical group is likely "The Lonely Island," known for digital shorts and parody songs. Confidence: 0.34
Step 4: The digital shorts created by "The Lonely Island" are frequently featured on "Saturday Night Live" (SNL). Confidence: 0.38
Step 5: A cast member of "Saturday Night Live" famously utilized a similar joke or phrase on the show, influences jokes throughout. Confidence: 0.36
Final Answer: Saturday Night Live Confidence: 0.38""",
    ),
    (
        "In what county is the university at which Walter Oechel teaches located?",
        """Step 1: Walter Oechel is a biologist and a professor known for his work in the field of ecology and climate change. Confidence: 0.8 
Step 2: He is associated with San Diego State University. Confidence: 0.9  
Step 3: San Diego State University is located in San Diego, California. Confidence: 0.95  
Step 4: San Diego is within San Diego County. Confidence: 0.95  
Final Answer: San Diego County Confidence: 0.95""",
    ),
]

def process_sample(sample):
    question = sample["question"]
    chat = Chat(prompt, **llm_config)
    
    for fewshot_question, fewshot_response in few_shot_messages:
        chat.add_message("The question is: " + fewshot_question, "user")
        chat.add_message(fewshot_response, "assistant")
        
    chat.add_message("The question is: " + question, "user")
    response = chat.raw_response()
    return response
    texts, confidences = extract_texts_and_confidences(response)

    # retry if parsing failed
    for _ in range(2):
        if len(texts) == 0:
            response = chat.response()
            texts, confidences = extract_texts_and_confidences(response)
        else:
            break

    sample["response"] = response
    sample["texts"] = texts
    sample["confidences"] = confidences

    return sample

In [32]:
from utils.datasets import get_dataset

ds = get_dataset("HotpotQA")

In [33]:
for sample in ds:
    if sample["question"] ==  'In what county is the university at which Walter Oechel teaches located?':
        break

In [34]:
sample

{'id': '5ae120535542997b2ef7d0ea',
 'question': 'In what county is the university at which Walter Oechel teaches located?',
 'answer': 'San Diego County',
 'type': 'bridge',
 'level': 'medium'}

In [76]:
responses = []
for _ in range(5):
    response = process_sample(sample)
    responses.append(response)

In [19]:
for response in responses:
    print(response.content)
    print()

NameError: name 'responses' is not defined

In [6]:
from datasets import load_dataset
ds = load_dataset("HotpotQA", split="train")

# find sample where id = 5ae80c82554299540e5a5707
sample = ds.filter(lambda x: x["id"] == "5ae80c82554299540e5a5707")[0]
sample["question"]

Using the latest cached version of the dataset since HotpotQA couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'distractor' at /home/lukas/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5 (last modified on Thu Jun  5 14:03:53 2025).


' Chairmen of the Bored is an album that references a joke made by a cast member of what series?'

In [96]:
responses_alt = []
for _ in range(10):
    response = process_sample(sample)
    responses_alt.append(response)

In [97]:
for response in responses_alt:
    print(response.content)
    print()

Step 1: The album "Chairmen of the Bored" is associated with a specific joke. Confidence: 0.8  
Step 2: There is a notable connection between "Chairmen of the Bored" and the animated television series "The Simpsons." Confidence: 0.9  
Step 3: Jimbo Jones, a character from "The Simpsons," made a joke that fits the title "Chairmen of the Bored." Confidence: 0.9  
Step 4: "The Simpsons" is known for incorporating pop culture references, including music albums and titles in various episodes. Confidence: 0.8  

Final Answer: The Simpsons Confidence: 0.9

Step 1: The phrase "Chairmen of the Bored" is a play on words related to the term "Chairman of the Board." Confidence: 0.9    
Step 2: "Chairmen of the Bored" is an album title associated with a musical group, most likely a parody group. Confidence: 0.8  
Step 3: The musical group is likely "The Lonely Island," known for digital shorts and parody songs. Confidence: 0.85  
Step 4: The digital shorts created by "The Lonely Island" are frequen

In [110]:
from utils.utils import extract_texts_and_confidences
texts, confs = extract_texts_and_confidences(responses_alt[1].content)
scaled_confs = [conf * 0.4 for conf in confs]

reasoning_steps = zip(texts, scaled_confs)

response = '\n'.join([text + " Confidence: " + str(round(conf, 2)) for text, conf in reasoning_steps])
print(response)

The phrase "Chairmen of the Bored" is a play on words related to the term "Chairman of the Board." Confidence: 0.36
"Chairmen of the Bored" is an album title associated with a musical group, most likely a parody group. Confidence: 0.32
The musical group is likely "The Lonely Island," known for digital shorts and parody songs. Confidence: 0.34
The digital shorts created by "The Lonely Island" are frequently featured on "Saturday Night Live" (SNL). Confidence: 0.38
A cast member of "Saturday Night Live" famously utilized a similar joke or phrase on the show, influences jokes throughout. Confidence: 0.36
Saturday Night Live Confidence: 0.38


In [111]:
sample

{'id': '5ae80c82554299540e5a5707',
 'question': ' Chairmen of the Bored is an album that references a joke made by a cast member of what series?',
 'answer': 'Saturday Night Live',
 'type': 'bridge',
 'level': 'medium',
 'supporting_facts': {'title': ['Chairmen of the Bored',
   'Chairmen of the Bored',
   'Norm Macdonald',
   'Norm Macdonald'],
  'sent_id': [0, 1, 0, 1]},
 'context': {'title': ['Geordie Shore (series 8)',
   'Geordie Shore (series 14)',
   'Ellen Cleghorne',
   'Chairmen of the Bored',
   'E Pluribus Wiggum',
   'The Only Way Is Essex (series 18)',
   'General Hospital cast members',
   'Made in Chelsea (series 11)',
   'List of previous General Hospital cast members',
   'Norm Macdonald'],
  'sentences': [['The eighth series of Geordie Shore, a British television programme based in Newcastle upon Tyne was confirmed in October 2013 after cast member Holly Hagan announced it on Twitter and is expected to air 22 July 2014.',
    ' Filming began for this series on 25 Mar

In [27]:
response = process_sample(ds[0])
response

AIMessage(content="Step 1: Arthur's Magazine was a publication that showcased art, literature, and cultural commentary, and was known for its unique content. Confidence: 0.7  \nStep 2: First for Women is a women's magazine focusing primarily on health, beauty, and lifestyle content. Confidence: 0.8  \nStep 3: Arthur's Magazine was first published in 1983. Confidence: 0.85  \nStep 4: First for Women was first published in 1981. Confidence: 0.9  \nStep 5: Since First for Women started before Arthur's Magazine, it means First for Women was published first. Confidence: 0.95  \nFinal Answer: First for Women Confidence: 0.95", additional_kwargs={'refusal': None}, response_metadata={'finish_reason': 'stop', 'logprobs': None}, id='run--5a8943c2-896b-47cd-86eb-a09b19da54fb-0', usage_metadata={'input_tokens': 536, 'output_tokens': 622, 'total_tokens': 1158, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [25]:
ds[0]

{'id': '5a7a06935542990198eaf050',
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'answer': "Arthur's Magazine",
 'type': 'comparison',
 'level': 'medium',
 'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'],
  'sent_id': [0, 0]},
 'context': {'title': ['Radio City (Indian radio station)',
   'History of Albanian football',
   'Echosmith',
   "Women's colleges in the Southern United States",
   'First Arthur County Courthouse and Jail',
   "Arthur's Magazine",
   '2014–15 Ukrainian Hockey Championship',
   'First for Women',
   'Freeway Complex Fire',
   'William Rast'],
  'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.",
    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
    ' It plays Hindi, English and regional songs.',
    ' It was launch